In [19]:
from __future__ import print_function
import datetime
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# If modifying these scopes, delete the token.json file
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

def get_calendar_events():
    creds = None
    # token.json stores the user's access/refresh tokens after first login
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If no (valid) creds, let user log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('calendar', 'v3', credentials=creds)

    # Call API to get upcoming events
    now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' = UTC time
    end = (datetime.datetime.utcnow() + datetime.timedelta(days=1)).isoformat() + 'Z'

    events_result = service.events().list(
        calendarId='primary', timeMin=now, timeMax=end,
        maxResults=10, singleEvents=True,
        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        return "No upcoming events found."
    
    output = []
    for event in events:
        title = event.get('summary', 'No Title')
        if title == 'Home':
            continue
        start = event['start'].get('dateTime', event['start'].get('date'))
        end = event['end'].get('dateTime', event['end'].get('date'))
        output.append(f"{start} - {end} — {title}")

    return " ".join(output)


In [50]:
from langchain.tools import BaseTool

class CalendarTool(BaseTool):
    name: str = "calendar_tool"
    description: str = "Get today's calendar events from Google Calendar."

    def _run(self, query: str):
        return get_calendar_events()

    def _arun(self, query: str):
        raise NotImplementedError("Async not supported")


# Slack

In [54]:
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
import os
import datetime
from datetime import timedelta

# Load your bot token
SLACK_BOT_TOKEN = os.getenv("SLACK_BOT_TOKEN")
client = WebClient(token=SLACK_BOT_TOKEN)

def get_recent_messages(channel_id: str, minutes_back: int = 60):
    try:
        # Timestamp for messages from X minutes ago until now
        now = datetime.datetime.utcnow()
        oldest = (now - timedelta(minutes=minutes_back)).timestamp()

        response = client.conversations_history(
            channel=channel_id,
            oldest=str(oldest)
        )

        messages = response["messages"]
        return [
            f"[{m['ts']}] {m.get('user', 'bot')} - {m.get('text', '')}"
            for m in messages
        ]
    except SlackApiError as e:
        return f"Error: {e.response['error']}"


In [51]:
from langchain.tools import BaseTool

class SlackTool(BaseTool):
    name: str = "slack_tool"
    description: str = "Fetch recent Slack messages"

    def _run(self, query: str):
        return "\n".join(get_recent_messages(channel_id="C0912N8JV8A", minutes_back=60))

    def _arun(self, query: str):
        raise NotImplementedError("Async not supported")


# Model

In [40]:
tools = [CalendarTool, SlackTool]

In [41]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4", model_provider="openai")

Enter API key for OpenAI:  ········


In [55]:
from langchain.agents import Tool, initialize_agent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4", temperature=0)

tools = [
    Tool(
        name="Calendar Tool",
        func=CalendarTool().run,
        description="Use this to get today's calendar events."
    ),
    Tool(
        name="Slack Tool",
        func=SlackTool().run,
        description="Use this to get recent Slack messages."
    ),
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

response = agent.run("What do I have on my calendar and what are the latest Slack updates?")
print(response)




> Entering new AgentExecutor chain...
I need to use both the Calendar Tool and the Slack Tool to answer this question.
Action: Calendar Tool
Action Input: {}

C:\Users\Vacuumlabs\AppData\Local\Temp\ipykernel_15088\3458627944.py:30: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' = UTC time
C:\Users\Vacuumlabs\AppData\Local\Temp\ipykernel_15088\3458627944.py:31: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end = (datetime.datetime.utcnow() + datetime.timedelta(days=1)).isoformat() + 'Z'



Observation: 2025-06-17T14:00:00+02:00 - 2025-06-17T15:05:00+02:00 — Kids 2025-06-18T09:30:00+02:00 - 2025-06-18T10:00:00+02:00 — Karol / Jucus - Vesna onboarding 2025-06-18T11:00:00+02:00 - 2025-06-18T11:30:00+02:00 — Karol / Janos sync (AUXM-3)
I have the calendar events, now I need to get the latest Slack updates.
Action: Slack Tool
Action Input: {}

C:\Users\Vacuumlabs\AppData\Local\Temp\ipykernel_15088\4276294009.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()



Observation: [1750150508.067049] U019UJU02UD - Hi summarizer, please finish AI academy by EOD
[1750150458.159129] U091TFPRUBW - <@U091TFPRUBW> has joined the channel
I now have both the calendar events and the latest Slack updates.
Final Answer: Your calendar events are: 
1. Kids from 14:00 to 15:05 on 2025-06-17
2. Karol / Jucus - Vesna onboarding from 09:30 to 10:00 on 2025-06-18
3. Karol / Janos sync (AUXM-3) from 11:00 to 11:30 on 2025-06-18

Your latest Slack updates are: 
1. U019UJU02UD - Hi summarizer, please finish AI academy by EOD
2. U091TFPRUBW - <@U091TFPRUBW> has joined the channel

> Finished chain.
Your calendar events are: 
1. Kids from 14:00 to 15:05 on 2025-06-17
2. Karol / Jucus - Vesna onboarding from 09:30 to 10:00 on 2025-06-18
3. Karol / Janos sync (AUXM-3) from 11:00 to 11:30 on 2025-06-18

Your latest Slack updates are: 
1. U019UJU02UD - Hi summarizer, please finish AI academy by EOD
2. U091TFPRUBW - <@U091TFPRUBW> has joined the channel
